In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
dataset=pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
dataset.head()

2024-05-22 14:10:44.224779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 14:10:44.224905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 14:10:44.371249: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [2]:
len(dataset['en'][1])

8

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
def embedder(text):
    input_ids = tokenizer.encode(text, return_tensors='tf')
    outputs = model(input_ids)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    #positional encodings
    angle_rates = 1 / np.power(10000, (2 * (768 // 2)) / np.float32(512))
    pos = np.arange(768)[:, np.newaxis]
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
    #adding positional encodings in the original encodings
    cls_embedding += tf.transpose(pos_encoding[:, :768, :][0])
    return cls_embedding
    

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
import math
def encoder(embed):
    # weights of query ,keys and values and other weights
    Wq=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wk=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wv=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W0=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W1=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.keras.activations.softmax(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    Key=normalized_embeddings*Wk
    Value=normalized_embeddings*Wv
    return [Key,Value]

In [5]:
def decoder(embed, arr):
    # weights of query ,keys and values and other weights
    Wq=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wk=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    Wv=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W0=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W1=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W01=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    W11=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    
    #Calculating query ,keys and values
    Query=embed*Wq
    Key=embed*Wk
    Value=embed*Wv
    # Scaled-Dot Product Attention
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W0*normalized_embeddings
    result1=tf.keras.activations.softmax(Layer1)
    result=W1*normalized_embeddings
    #Add layer
    embed+=result
    #normalize the embeddings
    embed = tf.keras.layers.LayerNormalization()(embed)
    #key ,value of encodings output
    Key,Value=arr
    Query=embed*Wq
    scores = tf.matmul(Query, Key, transpose_b=True) / math.sqrt(tf.cast(768, tf.float32))
    weights = tf.keras.activations.softmax(scores)
    result=tf.matmul(weights, Value)
    #Add layer
    embed+=result
    #normalize the embeddings
    normalized_embeddings = tf.keras.layers.LayerNormalization()(embed)
    #feed forward layer from scratch
    Layer1=W01*normalized_embeddings
    result1=tf.keras.activations.softmax(Layer1)
    result=W11*normalized_embeddings
    #Add layer
    embed+=result
    return embed

In [6]:
print(decoder(embedder(dataset['fr'][1]),encoder(embedder(dataset['en'][1]))))

tf.Tensor(
[[-2.01026511e+00  3.80490065e-01 -1.39400214e-01 -7.71663487e-01
   1.29558712e-01 -2.24051371e-01 -5.41466773e-02  2.05798283e-01
  -1.25549698e+00 -2.99427301e-01 -1.43110561e+00  2.25453898e-02
  -1.25714496e-01  6.51529312e-01 -1.08841822e-01  4.61091071e-01
  -5.48463345e-01 -2.39808321e-01  7.44036317e-01  3.95587971e-03
  -4.61858474e-02 -4.27888572e-01 -1.76005483e-01  4.37646247e-02
  -2.08050847e-01 -1.98394194e-01  2.48763680e-01  2.94221491e-01
  -9.35730100e-01  2.14390010e-01 -6.37565553e-02  9.36042815e-02
  -7.19696045e-01 -5.67512572e-01  6.01886034e-01  1.21587642e-01
   4.41611469e-01 -8.19133520e-02 -2.43117318e-01  9.33962539e-02
   4.83480036e-01  2.04100877e-01  3.08209389e-01 -4.91029441e-01
   1.19989328e-02 -3.20246723e-03 -5.38649273e+00  4.98544760e-02
  -2.22336817e+00  1.35519013e-01  2.55024731e-01  1.28354996e-01
   5.20406291e-03  5.51097631e-01  1.22859135e-01  1.01233077e+00
  -1.59312993e-01  9.35296476e-01  4.61312570e-02 -1.36187986e-01

In [7]:
def train(input1,output1):
    W0=tf.random.uniform(shape=(1, 768), minval=-0.5, maxval=0.5)
    encoder_output=encoder(embedder(input1))
    output_embed=embedder(output1)
    decoder_output=decoder(output_embed,encoder_output)
    Layer1=W0*decoder_output
    result=tf.nn.softmax(Layer1, axis=-1)
    predicted_token_ids = tf.expand_dims(result, axis=2)
    predicted_token_ids = tf.argmax(predicted_token_ids[0], axis=-1).numpy().flatten()
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids)
    predicted_text = tokenizer.convert_tokens_to_string(predicted_tokens)
    return "nothing"

In [8]:
print(train(dataset['en'][1],dataset['fr'][1]))

nothing
